### Задача 2
---
В пункте В2 сталь поступает на условный производственный комплекс, состоящий из сталелитейного и станкостроительного заводов. Он функционирует в течение 5 лет.

Начальный запас стали составляет 3100 т (для решения; интервал для исследования — 1000–10000 т (\*)).
Исходные производственные мощности заводов соответственно:
- по стали — 4300 т в год;
- по станкам — 1300 станков в год (для решения; интервал для исследования — 100–5000 станков). (\*\*)

Сталь расходуется на производство станков (1.1 т на каждый) и собственно стали, а также на расширение производственных мощностей комплекса. При этом каждая тонна стали, направленная на ее производство, обеспечивает выпуск 3.4 т. Тонна стали, идущая на расширение производственной мощности сталелитейного завода, увеличивает последнюю на 0.1 т, а для увеличения производственных мощностей станкостроительного завода на один станок необходимо затратить 15 т стали. Реализация решения о распределении стали на следующий год осуществляется в конце очередного года планируемого периода; станкостроительный завод не может получать более половины имеющегося запаса стали.

Требуется:
1. определить план распределения стали между сталелитейным и станкостроительным заводами, обеспечивающий выпуск максимального количества станков за плановый период;
1. провести исследование (поочередно, по одному фактору) влияния начальных условий (\*) и (\*\*) на структуру и результат решения (т.е. на распределение стали и на объем выпуска станков).


In [1]:
import numpy
from cvxopt.modeling import variable
from cvxopt.modeling import op

In [2]:
years = 5 # период
initial_steel = 6900 # начальный запас стали
production_capacity_steel = 2400 # производственная мощность литейки
production_capacity_machine = 600 # производственная мощность станков

# производство
steel_cost = 1 # потрачено на сталь
steel_rise = 2.7 # получено стали
machine_cost = 1.1 # потрачено на станок
machine_rise = 1 # получено станков

#pc - production capacity
# увеличение производственной мощности
steel_pc_cost = 1 # стоимость прироста мощности литейки
steel_pc_rise = 0.3 # получен прирост мощности литейки
machine_pc_cost = 5 # стоимость прироста мощности литейки
machine_pc_rise = 1 # получен прирост мощности литейки

In [3]:
spent_steel_prod = variable(years, 'spent_steel_prod') # потрачено на производство стали
spent_machine_prod = variable(years, 'spent_machine_prod') # потрачено на производство станков
spent_steel_inc = variable(years, 'spent_steel_inc') # потрачено на увеличение стали
spent_machine_inc = variable(years, 'spent_machine_inc') # потрачено на увеличение станков

In [4]:
get_steel = lambda year: spent_steel_prod[0] / steel_cost * steel_rise

In [5]:
# Ограничения по стали
cd = [
spent_steel_prod[0] + spent_machine_prod[0] + spent_steel_inc[0] + spent_machine_inc[0] == initial_steel,
spent_steel_prod[1] + spent_machine_prod[1] + spent_steel_inc[1] + spent_machine_inc[1] == steel_rise * spent_steel_prod[0],
spent_steel_prod[2] + spent_machine_prod[2] + spent_steel_inc[2] + spent_machine_inc[2] == steel_rise * spent_steel_prod[1],
spent_steel_prod[3] + spent_machine_prod[3] + spent_steel_inc[3] + spent_machine_inc[3] == steel_rise * spent_steel_prod[2],
spent_steel_prod[4] + spent_machine_prod[4] + spent_steel_inc[4] + spent_machine_inc[4] == steel_rise * spent_steel_prod[3],
]

In [6]:
# Ограничения на производстве стали
sd = [
spent_steel_prod[0] / steel_cost * steel_rise <= production_capacity_steel,
spent_steel_prod[1] / steel_cost * steel_rise <= production_capacity_steel + sum(spent_steel_inc[:1]) / steel_pc_cost * steel_pc_rise,
spent_steel_prod[2] / steel_cost * steel_rise <= production_capacity_steel + sum(spent_steel_inc[:2]) / steel_pc_cost * steel_pc_rise,
spent_steel_prod[3] / steel_cost * steel_rise <= production_capacity_steel + sum(spent_steel_inc[:3]) / steel_pc_cost * steel_pc_rise,
spent_steel_prod[4] / steel_cost * steel_rise <= production_capacity_steel + sum(spent_steel_inc[:4]) / steel_pc_cost * steel_pc_rise,
]

In [7]:
# Ограничения на производстве станков
md = [
spent_machine_prod[0] / machine_cost * machine_rise <= production_capacity_machine,
spent_machine_prod[1] / machine_cost * machine_rise <= production_capacity_machine + sum(spent_machine_inc[:1]) / machine_pc_cost * machine_pc_rise,
spent_machine_prod[2] / machine_cost * machine_rise <= production_capacity_machine + sum(spent_machine_inc[:2]) / machine_pc_cost * machine_pc_rise,
spent_machine_prod[3] / machine_cost * machine_rise <= production_capacity_machine + sum(spent_machine_inc[:3]) / machine_pc_cost * machine_pc_rise,
spent_machine_prod[4] / machine_cost * machine_rise <= production_capacity_machine + sum(spent_machine_inc[:4]) / machine_pc_cost * machine_pc_rise,
]

In [8]:
# Cтанкостроительный завод не может получать более половины имеющегося запаса стали
coef = 0.5
ad = [
spent_machine_prod[0] + spent_machine_inc[0] <= coef * initial_steel,
spent_machine_prod[1] + spent_machine_inc[1] <= coef * steel_rise * spent_steel_prod[0],
spent_machine_prod[2] + spent_machine_inc[2] <= coef * steel_rise * spent_steel_prod[1],
spent_machine_prod[3] + spent_machine_inc[3] <= coef * steel_rise * spent_steel_prod[2],
spent_machine_prod[4] + spent_machine_inc[4] <= coef * steel_rise * spent_steel_prod[3]
]

In [9]:
problem = op(
    -sum(spent_machine_prod/machine_cost),
    [
        # Ограничения по стали
        cd[0], cd[1], cd[2], cd[3], cd[4], 
        # Ограничения на производстве стали
        sd[0], sd[1], sd[2], sd[3], sd[4],        
        # Ограничения на производстве станков
        md[0], md[1], md[2], md[3], md[4],
        # Половины ограничений на производстве стали
        ad[0], ad[1], ad[2], ad[3], ad[4],
    ]
)        

In [10]:
problem.solve(solver = 'glpk')

In [11]:
problem.status

'optimal'

In [12]:
print(-problem.objective.value()[0])

5370.107378962964


In [13]:
print(spent_steel_prod.name, [spent_steel_prod.value[i] for i in range(5)])
print(spent_machine_prod.name, [spent_machine_prod.value[i] for i in range(5)])
print(spent_steel_inc.name, [spent_steel_inc.value[i] for i in range(5)])
print(spent_machine_inc.name, [spent_machine_inc.value[i] for i in range(5)])

spent_steel_prod [888.8888888888888, 1173.4567901234566, 1176.4060356652947, 1026.91668903155, 1284.0718234561803]
spent_machine_prod [660.0, 1273.8000000000002, 1257.5640000000003, 1329.4165866666665, 1386.3375301925926]
spent_steel_inc [2561.1111111111113, 26.543209876543482, 407.76063100137173, 561.231459116598, 102.26570673641235]
spent_machine_inc [2790.0, -73.80000000000021, 326.60266666666627, 258.7315614814814, 0.0]
